In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [5]:
# defining a transformation to convert images to tensor
transform = transforms.ToTensor()

In [7]:
# downloading and loading the CIFAR-10 training and test datasets
# first time when we run this, it will download the dataset to our computer
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# defining the 10 classes in the dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print("CIFAR-10 dataset loaded!")

Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 dataset loaded!


In [11]:
import torch.nn as nn
import torch.nn.functional as F

# defining a Convolutional Neural Network
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # first convolutional layer: takes 3 input channels (for RGB images), outputs 6, with a 5x5 filter
        self.conv1 = nn.Conv2d(3, 6, 5)
        # first pooling layer: a 2x2 max pooling operation
        self.pool = nn.MaxPool2d(2, 2)
        # second convolutional layer: takes 6 input channels, outputs 16, with a 5x5 filter
        self.conv2 = nn.Conv2d(6, 16, 5)
        # fully connected (linear) layers
        # the input size (16 * 5 * 5) is calculated based on the output of the last pooling layer
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10 is for our 10 output classes (0-9)

    def forward(self, x):
        # forward pass through the network
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # reshape the output from convolutional layers to a 1D vector for the linear layers
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# creating an instance of our CNN
cnn_model = SimpleCNN()
print("---- Our CNN architecture ----")
print(cnn_model)

---- Our CNN architecture ----
SimpleCNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [16]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# make sure that we have the SimpleCNN class and data loaders from the previous cell in this notebook and we might want to re-run those cells

# creating an instance of our CNN
cnn_model = SimpleCNN()

# defining the loss function and optimizer
# CrossEntropyLoss is the standard loss function for classification problems
criterion = nn.CrossEntropyLoss()
# we will use the Adam optimizer, which is more advanced and often performs better than SGD
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# the training loop
num_epochs = 1
for epoch in range(num_epochs): # we will start with just 1 epoch to save time
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # getting the inputs and labels from our data loader
        inputs, label = data

        # clearing previous gradients
        optimizer.zero_grad()

        # forward pass: making a prediction
        outputs = model_cnn(inputs)
        # calculate loss
        loss = criterion(outputs, label)

        # backward pass: compute gradients
        loss.backward()
        # optimizer step: update weights
        optimizer.step()

        # printing statistics
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print("Finished Training")

[1, 2000] loss: 2.304
[1, 4000] loss: 2.305
[1, 6000] loss: 2.305
[1, 8000] loss: 2.306
[1, 10000] loss: 2.305
[1, 12000] loss: 2.304
Finished Training


In [20]:
# switching the model to evaluation mode
cnn_model.eval()

# disabling the gradient calculation for evaluation
with torch.no_grad():
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        # make a prediction
        outputs = cnn_model(images)
        # get the predicted class by finding the index of the max log-probability
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%")

Accuracy of the network on the 10000 test images: 10.00%
